## Collaborative Filtering
Collaborative filtering is a machine learning technique that predicts ratings awarded to items by users.

### Import the ALS class
In this exercise, you will use the Alternating Least Squares collaborative filtering algorithm to creater a recommender.

In [2]:
from pyspark.ml.recommendation import ALS

### Load Source Data
The source data for the recommender is in two files - one containing numeric IDs for movies and users, along with user ratings; and the other containing details of the movies.

In [4]:
ratings = spark.read.csv('wasb://spark@dbshepstor.blob.core.windows.net/data/ratings.csv', inferSchema=True, header=True)
movies = spark.read.csv('wasb://spark@dbshepstor.blob.core.windows.net/data/movies.csv', inferSchema=True, header=True)
ratings.join(movies, "movieId").show()

+-------+------+------+----------+--------------------+--------------------+
movieId|userId|rating| timestamp| title| genres|
+-------+------+------+----------+--------------------+--------------------+
 31| 1| 2.5|1260759144|Dangerous Minds (...| Drama|
 1029| 1| 3.0|1260759179| Dumbo (1941)|Animation|Childre...|
 1061| 1| 3.0|1260759182| Sleepers (1996)| Thriller|
 1129| 1| 2.0|1260759185|Escape from New Y...|Action|Adventure|...|
 1172| 1| 4.0|1260759205|Cinema Paradiso (...| Drama|
 1263| 1| 2.0|1260759151|Deer Hunter, The ...| Drama|War|
 1287| 1| 2.0|1260759187| Ben-Hur (1959)|Action|Adventure|...|
 1293| 1| 2.0|1260759148| Gandhi (1982)| Drama|
 1339| 1| 3.5|1260759125|Dracula (Bram Sto...|Fantasy|Horror|Ro...|
 1343| 1| 2.0|1260759131| Cape Fear (1991)| Thriller|
 1371| 1| 2.5|1260759135|Star Trek: The Mo...| Adventure|Sci-Fi|
 1405| 1| 1.0|1260759203|Beavis and Butt-H...|Adventure|Animati...|
 1953| 1| 4.0|1260759191|French Connection...|Action|Crime|Thri...|
 2105| 1| 4.0|1260759139| Tron (1982)|Action|Adventure|...|
 2150| 1| 3.0|1260759194|Gods Must Be Craz...| Adventure|Comedy|
 2193| 1| 2.0|1260759198| Willow (1988)|Action|Adventure|...|
 2294| 1| 2.0|1260759108| Antz (1998)|Adventure|Animati...|
 2455| 1| 2.5|1260759113| Fly, The (1986)|Drama|Horror|Sci-...|
 2968| 1| 1.0|1260759200| Time Bandits (1981)|Adventure|Comedy|...|
 3671| 1| 3.0|1260759117|Blazing Saddles (...| Comedy|Western|
+-------+------+------+----------+--------------------+--------------------+
only showing top 20 rows

### Prepare the Data
To prepare the data, split it into a training set and a test set.

In [6]:
data = ratings.select("userId", "movieId", "rating")
splits = data.randomSplit([0.7, 0.3])
train = splits[0].withColumnRenamed("rating", "label")
test = splits[1].withColumnRenamed("rating", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 70034 Testing Rows: 29970

### Build the Recommender
The ALS class is an estimator, so you can use its **fit** method to traing a model, or you can include it in a pipeline. Rather than specifying a feature vector and as label, the ALS algorithm requries a numeric user ID, item ID, and rating.

In [8]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="label")
model = als.fit(train)

### Test the Recommender
Now that you've trained the recommender, you can see how accurately it predicts known ratings in the test set.

In [10]:
prediction = model.transform(test)
prediction.join(movies, "movieId").select("userId", "title", "prediction", "trueLabel").show(100, truncate=False)

+------+--------------------------------+----------+---------+
userId|title |prediction|trueLabel|
+------+--------------------------------+----------+---------+
575 |Awfully Big Adventure, An (1995)|NaN |4.0 |
232 |Guilty as Sin (1993) |2.6910245 |4.0 |
460 |Hudsucker Proxy, The (1994) |4.3136444 |5.0 |
274 |Hudsucker Proxy, The (1994) |5.1962156 |5.0 |
292 |Hudsucker Proxy, The (1994) |4.205546 |3.5 |
306 |Hudsucker Proxy, The (1994) |4.1447997 |3.0 |
659 |Hudsucker Proxy, The (1994) |4.3137717 |4.0 |
502 |Hudsucker Proxy, The (1994) |5.484579 |4.0 |
23 |Hudsucker Proxy, The (1994) |3.3261085 |3.5 |
380 |Hudsucker Proxy, The (1994) |4.128973 |4.0 |
624 |Hudsucker Proxy, The (1994) |3.0916016 |4.0 |
585 |Hudsucker Proxy, The (1994) |4.4571357 |4.0 |
73 |Hudsucker Proxy, The (1994) |4.1143765 |4.0 |
195 |Hudsucker Proxy, The (1994) |3.6290503 |3.0 |
487 |Hudsucker Proxy, The (1994) |3.7503219 |4.0 |
574 |Hudsucker Proxy, The (1994) |4.4702873 |3.5 |
468 |Hudsucker Proxy, The (1994) |2.9303226 |4.0 |
521 |Hudsucker Proxy, The (1994) |3.7380652 |3.5 |
497 |What Happened Was... (1994) |NaN |2.0 |
547 |What Happened Was... (1994) |NaN |3.0 |
509 |What Happened Was... (1994) |NaN |3.0 |
296 |High School High (1996) |1.4095564 |4.5 |
516 |Dirty Dancing (1987) |3.0079699 |3.0 |
133 |Dirty Dancing (1987) |2.4565787 |1.5 |
372 |Dirty Dancing (1987) |2.6509361 |4.0 |
500 |Dirty Dancing (1987) |3.7018418 |4.0 |
306 |Dirty Dancing (1987) |3.6197517 |4.0 |
19 |Dirty Dancing (1987) |2.0967772 |3.0 |
531 |Dirty Dancing (1987) |2.5262642 |5.0 |
505 |Dirty Dancing (1987) |3.0720944 |4.0 |
387 |Dirty Dancing (1987) |3.800532 |4.0 |
418 |Dirty Dancing (1987) |5.2994494 |5.0 |
303 |Dirty Dancing (1987) |3.2639213 |3.0 |
574 |Dirty Dancing (1987) |3.255555 |3.0 |
105 |Dirty Dancing (1987) |2.702092 |3.0 |
575 |Dirty Dancing (1987) |2.7130492 |3.0 |
242 |Dirty Dancing (1987) |3.7595 |5.0 |
294 |Dirty Dancing (1987) |3.3245168 |3.5 |
311 |Dirty Dancing (1987) |3.171254 |5.0 |
232 |Local Hero (1983) |4.014909 |4.0 |
564 |Local Hero (1983) |2.9136329 |3.0 |
580 |Candyman (1992) |2.3636613 |3.0 |
102 |Candyman (1992) |3.952166 |3.0 |
457 |Candyman (1992) |2.2570097 |2.5 |
463 |Men in Black (a.k.a. MIB) (1997)|3.7844343 |4.0 |
471 |Men in Black (a.k.a. MIB) (1997)|3.4600024 |3.0 |
243 |Men in Black (a.k.a. MIB) (1997)|3.5351605 |3.0 |
597 |Men in Black (a.k.a. MIB) (1997)|3.911614 |5.0 |
155 |Men in Black (a.k.a. MIB) (1997)|1.9783411 |5.0 |
211 |Men in Black (a.k.a. MIB) (1997)|4.3561106 |3.0 |
412 |Men in Black (a.k.a. MIB) (1997)|3.9505098 |3.0 |
501 |Men in Black (a.k.a. MIB) (1997)|4.0485797 |4.0 |
253 |Men in Black (a.k.a. MIB) (1997)|4.3071375 |3.5 |
350 |Men in Black (a.k.a. MIB) (1997)|3.7895494 |4.0 |
417 |Men in Black (a.k.a. MIB) (1997)|3.6134193 |4.0 |
285 |Men in Black (a.k.a. MIB) (1997)|3.5331686 |4.0 |
232 |Men in Black (a.k.a. MIB) (1997)|4.2320285 |5.0 |
246 |Men in Black (a.k.a. MIB) (1997)|4.472905 |5.0 |
367 |Men in Black (a.k.a. MIB) (1997)|3.0598114 |3.0 |
111 |Men in Black (a.k.a. MIB) (1997)|4.1169667 |4.0 |
603 |Men in Black (a.k.a. MIB) (1997)|4.0991855 |5.0 |
212 |Men in Black (a.k.a. MIB) (1997)|3.200346 |1.5 |
442 |Men in Black (a.k.a. MIB) (1997)|4.117409 |4.5 |
328 |Men in Black (a.k.a. MIB) (1997)|3.2866192 |4.0 |
500 |Men in Black (a.k.a. MIB) (1997)|3.0750167 |0.5 |
402 |Men in Black (a.k.a. MIB) (1997)|3.661776 |3.5 |
120 |Men in Black (a.k.a. MIB) (1997)|4.215155 |4.0 |
48 |Men in Black (a.k.a. MIB) (1997)|3.533523 |3.5 |
268 |Men in Black (a.k.a. MIB) (1997)|4.010483 |4.0 |
430 |Men in Black (a.k.a. MIB) (1997)|3.5617814 |5.0 |
560 |Men in Black (a.k.a. MIB) (1997)|4.181435 |3.0 |
607 |Men in Black (a.k.a. MIB) (1997)|4.2329006 |4.5 |
15 |Men in Black (a.k.a. MIB) (1997)|3.1849499 |4.0 |
262 |Men in Black (a.k.a. MIB) (1997)|2.164331 |2.0 |
358 |Men in Black (a.k.a. MIB) (1997)|3.186699 |3.0 |
523 |Men in Black (a.k.a. MIB) (1997)|4.381627 |4.0 |
547 |Men in Black (a.k.a. MIB) (1997)|2.6052136 |2.0 |
173 |Men in Bl

The data used in this exercise describes 5-star rating activity from [MovieLens](http://movielens.org), a movie recommendation service. It was created by GroupLens, a research group in the Department of Computer Science and Engineering at the University of Minnesota, and is used here with permission.

This dataset and other GroupLens data sets are publicly available for download at <http://grouplens.org/datasets/>.

For more information, see F. Maxwell Harper and Joseph A. Konstan. 2015. [The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015)](http://dx.doi.org/10.1145/2827872)